In [23]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
df = pd.read_csv("data/cash_deposit_prediction.csv")
df.head()

,Serial Number,Main Office,Branch Number,Established Date,Acquired Date,City,County,State,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
0,1,0,2866,02/06/1910,11/13/2004,Wales,Waukesha,WI,21386.0,23981.0,24825.0,26908.0,30681.0,30680.0,37564
1,2,0,2867,02/28/1910,11/13/2004,Germantown,Washington,WI,55454.0,56564.0,64732.0,73523.0,81357.0,89270.0,106177
2,3,0,2868,05/06/1907,11/13/2004,Brookfield,Waukesha,WI,90882.0,104300.0,125038.0,125906.0,132501.0,137363.0,145054
3,4,0,2869,06/20/1936,11/13/2004,Pewaukee,Waukesha,WI,45674.0,49288.0,53251.0,70402.0,74742.0,73837.0,85996
4,5,0,2870,04/22/1963,11/13/2004,Waukesha,Waukesha,WI,64181.0,72217.0,69920.0,80703.0,75971.0,73225.0,81706


In [14]:
df.columns = map(str.lower,df.columns)
df.columns = df.columns.str.replace(" ","_")
df = df.replace("NaN",np.nan)
df.head()

,serial_number,main_office,branch_number,established_date,acquired_date,city,county,state,2010_deposits,2011_deposits,2012_deposits,2013_deposits,2014_deposits,2015_deposits,2016_deposits
0,1,0,2866,02/06/1910,11/13/2004,Wales,Waukesha,WI,21386.0,23981.0,24825.0,26908.0,30681.0,30680.0,37564
1,2,0,2867,02/28/1910,11/13/2004,Germantown,Washington,WI,55454.0,56564.0,64732.0,73523.0,81357.0,89270.0,106177
2,3,0,2868,05/06/1907,11/13/2004,Brookfield,Waukesha,WI,90882.0,104300.0,125038.0,125906.0,132501.0,137363.0,145054
3,4,0,2869,06/20/1936,11/13/2004,Pewaukee,Waukesha,WI,45674.0,49288.0,53251.0,70402.0,74742.0,73837.0,85996
4,5,0,2870,04/22/1963,11/13/2004,Waukesha,Waukesha,WI,64181.0,72217.0,69920.0,80703.0,75971.0,73225.0,81706


In [15]:
from sklearn.model_selection import train_test_split
df.set_index(keys='serial_number',inplace=True,drop=True)

df["established_date"] = pd.to_datetime(df["established_date"])
df["acquired_date"] = pd.to_datetime(df["acquired_date"])
y = df["2016_deposits"]
X = df.drop(["2016_deposits"],axis=1)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state=3)

## Generate a feature of age of funds from inception date

In [16]:
time_col = ['established_date', 'acquired_date']

# Code starts here
for col_name in time_col:
    new_col_name = "since_"+col_name
    X_train[new_col_name] = (pd.datetime.now() - X_train[col_name])
    X_train[new_col_name] = X_train[new_col_name] / np.timedelta64(1, 'Y')
    X_train.drop(col_name,axis=1,inplace=True)
    
for col_name in time_col:
    new_col_name = "since_"+col_name
    X_val[new_col_name] = (pd.datetime.now() - X_val[col_name])
    X_val[new_col_name] = X_val[new_col_name] / np.timedelta64(1, 'Y')
    X_val.drop(col_name,axis=1,inplace=True)
X_train.head()

,main_office,branch_number,city,county,state,2010_deposits,2011_deposits,2012_deposits,2013_deposits,2014_deposits,2015_deposits,since_established_date,since_acquired_date
serial_number,,,,,,,,,,,,,
2606,0,6509,Highland,San Bernardino,CA,19374.0,23293.0,26564.0,28966.0,33219.0,38892.0,14.679686,12.467457
2001,0,5415,Melbourne,Brevard,FL,33042.0,34507.0,39912.0,65500.0,74012.0,76850.0,19.366983,12.467457
2768,0,6774,Plano,Collin,TX,8855.0,18624.0,24435.0,54422.0,63150.0,79338.0,12.259376,NaN
1724,0,4969,San Clemente,Orange,CA,109197.0,132822.0,127834.0,132220.0,146477.0,159482.0,43.200463,12.467457
1120,0,4317,Morris,Grundy,IL,14274.0,15475.0,18996.0,22121.0,26961.0,30652.0,13.297043,NaN


In [17]:
from sklearn.preprocessing import LabelEncoder
cat = X_train.select_dtypes(include='O').columns.tolist()

X_train.fillna(0,inplace=True)
X_val.fillna(0,inplace=True)
le = LabelEncoder()
for x in cat:
    X_train[x] = le.fit_transform(X_train[x])
    X_val[x] = le.fit_transform(X_val[x])


X_train_temp = pd.get_dummies(data = X_train, columns = cat)
X_val_temp = pd.get_dummies(data = X_val, columns = cat)

# Shape of train and test data
print(X_train_temp.shape,X_val_temp.shape)

(2842, 1552) (948, 800)


In [21]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

dt = DecisionTreeRegressor(random_state=5)
dt.fit(X_train,y_train)
accuracy = dt.score(X_val,y_val)
y_pred = dt.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val,y_pred))
print(accuracy,rmse)

0.9333991141166543 68372.52930639003


In [20]:
from xgboost import XGBRegressor

xgb = XGBRegressor(max_depth=50,learning_rate=0.83,n_estimators=100)
xgb.fit(X_train,y_train)
accuracy = xgb.score(X_val,y_val)
y_pred = xgb.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val,y_pred))
print(accuracy,rmse)


[15:32:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.7611949022857951 129468.29829718929
